# Computational Considerations in LA for Data Scientsts

**Purpose:** The purpose of this workbook is to help you get comfortable with the topics outlined below.

**Prereqs**
* Python Fundamentals Workbook or a good grasp of basic Python
* Numpy Workbook or a good grasp of creating and manipulating numpy arrays
* Matplotlib Workbook or a good grasp of plotting using matplotlib
    
**Recomended Usage**
* Run each of the cells (Shift+Enter) and edit them as necessary to solidify your understanding
* Do any of the exercises that are relevant to helping you understand the material

**Topics Covered**
* Computational resource monitoring (speed and memory)
* Optimizations (noteworthy library optimizations and other optimization options)

# Workbook Setup

In [2]:
# Reload all modules before executing a new line
%load_ext autoreload
%autoreload 2

# Abide by PEP8 code style
# %load_ext pycodestyle_magic
# %pycodestyle_on

# Plot all matplotlib plots in output cell and save on close
%matplotlib inline

In [3]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D

from IPython.display import YouTubeVideo

## Helper Functions

In [3]:
def print_a(a):
    print('Dim: {}\nShape: {}\n{}'.format(a.ndim, a.shape, a))

In [4]:
def plot_2d_vectors(vectors=None, labels=None, colors=None, title=None):
    '''A custom vector plotting function that makes it easy to plot 2D vectors'''

    # TODO ADD KWARGS THAT CAN BE PASSED INTO THE SET FUNCTION

    # TODO input data checks
    # type(vectors) -> list of int tuple
    # type(vector_labels) -> list of strings
    # len(vectors) == len(vector_labels)
    # title == None | string
    # etc
    
    fig, ax = plt.subplots(1)

    origin = [0], [0]
    xmin, xmax = 0, 0
    ymin, ymax = 0, 0

    if colors == None:
        color_list = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
        colors = np.random.choice(color_list, len(vectors))

    if labels == None:
        labels = ['V'+str(i) for i in range(len(vectors))]

    for i in range(len(vectors)):
        ax.quiver(*origin, *vectors[i], color=colors[i], label=labels[i], units='xy', scale=1)

        if vectors[i][0] < xmin:
            xmin = vectors[i][0]
        elif vectors[i][0] > xmax:
            xmax = vectors[i][0]

        if vectors[i][1] < ymin:
            ymin = vectors[i][1]
        elif vectors[i][1] > ymax:
            ymax = vectors[i][1]

    buffer = 2
    plt.xlim(xmin - buffer, xmax + buffer)
    plt.ylim(ymin - buffer, ymax + buffer)

    plt.grid()

    if title != None:
        plt.title(title)

    plt.legend()

In [5]:
def plot_3d_vectors(vectors=None, labels=None, colors=None, title=None):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    origin = [0], [0], [0]
    xmin, xmax = 0, 0
    ymin, ymax = 0, 0
    zmin, zmax = 0, 0

    if colors == None:
        color_list = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
        colors = np.random.choice(color_list, len(vectors))

    if labels == None:
        labels = ['V'+str(i) for i in range(len(vectors))]

    for i in range(len(vectors)):
        ax.quiver3D(*origin, *vectors[i], color=colors[i], label=labels[i])

        if vectors[i][0] < xmin:
            xmin = vectors[i][0]
        elif vectors[i][0] > xmax:
            xmax = vectors[i][0]

        if vectors[i][1] < ymin:
            ymin = vectors[i][1]
        elif vectors[i][1] > ymax:
            ymax = vectors[i][1]

        if vectors[i][2] < zmin:
            zmin = vectors[i][2]
        elif vectors[i][2] > zmax:
            zmax = vectors[i][2]

    print('xlim: {}, {}'.format(xmin, xmax))
    print('ylim: {}, {}'.format(ymin, ymax))
    print('zlim: {}, {}'.format(zmin, zmax))

    buffer = 2
    ax.set_xlim([xmin - buffer, xmax + buffer])
    ax.set_ylim([ymin - buffer, ymax + buffer])
    ax.set_zlim([zmin - buffer, zmax + buffer])
    
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')

    plt.grid()

    if title != None:
        plt.title(title)

    plt.legend()

# Computational Linear Algebra Considerations

Computational linear algebra calculations on large datasets can be time and memoty intensive, here are some techniques help monitor computational resources.

Optimizing a computer to perform calculations means we need to learn to optimize their memory efficieny and speed. Luckily, libraries take care of most of this for us but it helps to be mindful of what they are doing. In this section we are going to just skim the surface..

Memory Usage Tools

In [ ]:
import psutil

In [ ]:
process = psutil.Process(os.getpid())
t = process.memory_info()

In [ ]:
t.vms, t.rss

In [ ]:
def mem_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / psutil.virtual_memory().total

In [ ]:
mem_usage()

Timing Tools

In [ ]:
%%timeit

## Broadcasting

In order to perform matrix operations quickly matrix libraries use broadcasting in which the calculations are performed more efficiently and in C rather than Python.

### General Broadcasting Rules

NumPy compares array shapes element-wise (starting with the trailing dims)

Two dimensions are compatible when

they are equal, or
one of them is 1

```
Image  (3d array): 256 x 256 x 3
Scale  (1d array):             3
Result (3d array): 256 x 256 x 3
```

When either of the dimensions compared is one, the other is used. In other words, dimensions with size 1 are stretched or “copied” to match the other.

```
A      (4d array):  8 x 1 x 6 x 1
B      (3d array):      7 x 1 x 5
Result (4d array):  8 x 7 x 6 x 5
```

Element-wise operation for same dims

In [6]:
a = np.array([1.0, 2.0, 3.0])
b = np.array([2.0, 2.0, 2.0])
a * b

array([2., 4., 6.])

Scalar spread across dims

In [7]:
a = np.array([1.0, 2.0, 3.0])
b = 2.0
a * b

array([2., 4., 6.])

In [18]:
x = np.arange(4)
print(x)
print(x.shape)

[0 1 2 3]
(4,)


In [19]:
y = np.ones(5)
print(y)
print(y.shape)

[1. 1. 1. 1. 1.]
(5,)


In [17]:
x + y

ValueError: operands could not be broadcast together with shapes (4,) (5,) 

In [21]:
xx = x.reshape(4, 1)
print(xx)
print(xx.shape)

[[0]
 [1]
 [2]
 [3]]
(4, 1)


In [22]:
xx + y

array([[1., 1., 1., 1., 1.],
       [2., 2., 2., 2., 2.],
       [3., 3., 3., 3., 3.],
       [4., 4., 4., 4., 4.]])

In [24]:
z = np.ones((3, 4))
print(z)
print(z.shape)

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
(3, 4)


In [25]:
x + z

array([[1., 2., 3., 4.],
       [1., 2., 3., 4.],
       [1., 2., 3., 4.]])

There are several methods for dimensionality reduction....

* **Factorization / Decomposition:** We could factor the matrix, or break it down into the product of lower dimensional matrices.
* PCA ...

## Dimensionality Reduction

Frequently data scientists will work with what are called sparse matrices (matrices with lots of zeros). In order to more efficiently store sparse matrices we can store just the non-zero values.

*I highly recoment checking out the Principle Component Analysis Workbook for more details on dimensionality reduction if its relevant to you.*

Some data structures better suited for efficiently working with sparse matrices:

* Dictionary of Keys. A dictionary is used where a row and column index is mapped to a value.

* List of Lists. Each row of the matrix is stored as a list, with each sublist containing the column index and the value.

* Coordinate List. A list of tuples is stored with each tuple containing the row index, column index, and the value.

* Compressed Sparse Row. The sparse matrix is represented using three one-dimensional arrays for the non-zero values, the extents of the rows, and the column indexes.

* Compressed Sparse Column. The same as the Compressed Sparse Row method except the column indices are compressed and read first before the row indices.

In [29]:
from scipy.sparse import csr_matrix

In [30]:
# create dense matrix
A = np.array([[1, 0, 0, 1, 0, 0], [0, 0, 2, 0, 0, 1], [0, 0, 0, 2, 0, 0]])
print(A)

[[1 0 0 1 0 0]
 [0 0 2 0 0 1]
 [0 0 0 2 0 0]]


In [31]:
# convert to sparse matrix (CSR method)
S = csr_matrix(A)
print(S)

  (0, 0)	1
  (0, 3)	1
  (1, 2)	2
  (1, 5)	1
  (2, 3)	2


In [32]:
# reconstruct dense matrix
B = S.todense()
print(B)

[[1 0 0 1 0 0]
 [0 0 2 0 0 1]
 [0 0 0 2 0 0]]


### Clustering

### <span style="color:red;">Singular Value Decomposition (SVD)</span>

In [74]:
M = np.array([[3, 0, 2], [2, 0, -2], [0, 1, 1]])
M

array([[ 3,  0,  2],
       [ 2,  0, -2],
       [ 0,  1,  1]])

In [75]:
U, S, Vtranspose = np.linalg.svd(M)
print(U)
print(S)
print(Vtranspose)

[[-0.95123459  0.23048583 -0.20500982]
 [-0.28736244 -0.90373717  0.31730421]
 [-0.11214087  0.36074286  0.92589903]]
[3.72021075 2.87893436 0.93368567]
[[-0.9215684  -0.03014369 -0.38704398]
 [-0.38764928  0.1253043   0.91325071]
 [ 0.02096953  0.99166032 -0.12716166]]


## Additional Resources

[Linear Algebra Playlist (by 1blue3brown)](https://www.youtube.com/playlist?list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab)

[Understanding Linearity and Linear Transformations (by Khan Academy)](https://www.khanacademy.org/math/linear-algebra/matrix-transformations/linear-transformations/a/visualizing-linear-transformations)

[Numerical Linear Algebra (by Rachel Thomas, Fastai)](https://nbviewer.jupyter.org/github/fastai/numerical-linear-algebra/tree/master/)